### [ 타이타닉 분석 및 시각화 ]
1. 생존자별 연령
2. 생존자의 성별
3. 생존자들의 연령과 성별
4. 등석별 생존자 비교

[1] 모듈 준비 및 데이터 로딩<hr>

In [216]:
## 모듈 로딩
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib

In [217]:
## 데이터 준비
# DATA_FILE = '../Data/titanic.csv'
DATA_FILE = '../Data/titanic_train.csv'

## 데이터 로딩
dataDF = pd.read_csv(DATA_FILE)

## 복사본 준비
dataDF2 = dataDF.copy()

[2] 기본 데이터 확인 및 전처리 방향 설정 <hr>


In [218]:
## -------------------------------------------------------
## [2-1] 기본 데이터 확인
## -------------------------------------------------------
## 실제 데이터
display(dataDF2.head())
## 데이터 전반 정보' : 행/샘플 수, 컬럼수, 컬측치, 컬럼별 타입
dataDF2.info()
## 컬럼별 상세 => 데이터 분포 및 의미기반 타입
numDF = dataDF2.describe()
objDF = dataDF2.describe(exclude=['number']) ## 해당 타입을 제외함
display('수치형 컬럼들' , numDF)
display('텍스트형 컬럼',objDF)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


'수치형 컬럼들'

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


'텍스트형 컬럼'

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,G6,S
freq,1,577,7,4,644


In [219]:
## ----------------------------------------
## [2-2] 전처리 방향 설정
## ----------------------------------------
## -> 결측치 처리 컬럼
##    Age               -> 1).버림/추가 수집    2).대체 여부

## -> 자료형 변환 설정
## Sex          object  ->  category
## Survived     int64   ->  category
## Plcass       int64   ->  category
## Embarked     object  ->  category
## SibSp/Parch	int64   ->  고유값 체크 필요함

## -> 불필요한 컬럼 : 삭제
## -> 유사/동일한 데이터 컬럼, 대부분이 결측치인 데이터 컬럼, 분석 목적과 맞지 않는 컬럼
##      -> PassengerId, Name, Cabin, Embarked

[3] 전처리 진행 <hr>

In [220]:
## ----------------------------------------------------
## [3-1] 불필요한 컬럼 삭제 : drop() : 삭제 방향
##       -> PassengerId, Name, Cabin, Embarked
## ----------------------------------------------------
del_col=['PassengerId', 'Name', 'Cabin', 'Embarked']
dataDF2.drop(labels=del_col, axis=1)
dataDF2.drop(columns=del_col, inplace=True)
print('정리 후')
print(dataDF2.columns)

정리 후
Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare'], dtype='object')


In [221]:
## --------------------------------------------------------------------------
## [3-2] 컬럼별 자료형 변환 : astype( 타입명 ) -> inplace 없음
##      - 기준 : 연산가능여부 -> 수치형/범주형
##      - 목적 : 분석목적/주제/시각화 단계에서 수치형 => 범주형
## --------------------------------------------------------------------------
## 타입 유지!! 
for col in ['SibSp','Parch'] :
    print(f'[{col}] -------------- {dataDF2[col].nunique()}')
    print(f'{dataDF2[col].unique()}')

for col in ['Survived', 'Pclass', 'Sex'] :
    dataDF2[col] = dataDF2[col].astype('category')

## 확인
dataDF2.info()


[SibSp] -------------- 7
[1 0 3 4 2 5 8]
[Parch] -------------- 7
[0 1 2 5 3 4 6]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Survived  891 non-null    category
 1   Pclass    891 non-null    category
 2   Sex       891 non-null    category
 3   Age       714 non-null    float64 
 4   SibSp     891 non-null    int64   
 5   Parch     891 non-null    int64   
 6   Ticket    891 non-null    object  
 7   Fare      891 non-null    float64 
dtypes: category(3), float64(2), int64(2), object(1)
memory usage: 37.9+ KB


In [222]:
## --------------------------------------------------------------------------------
## [3-3] 결측치(Missing Value) : 빈칸 / 데이터 없는 것
## - 표기법 : NaN, NaT, NA, null, None
## - 결측치 조사 => isna() / isnull() : boolean값,   notna()/notnull() : boolean 값
## - 결측치 삭제 => [삭제] dropna()     [대체] fillna(), ffill(), bfill() 
## --------------------------------------------------------------------------------
## - 가장 좋은 방법 : 다시 수집 => 시간/비용
## - 일부/미미한 수 : 삭제
## - 대체 : 같은 행의 다른 컬럼들과 비슷한 행들의 값과 비교해서 채우기
##          같은 컬럼의 패턴/흐름 비슷하다면 이전/이후값으로 채우기
##          결측치를 위한 타입 (예 : 기타, 응답없음, 생각해본적없음, 미상, 알수없음) 즉 생성
## ----------------------------------------------------------------------------------
## => 결측치 조사
naSR = dataDF2.isna().sum()

## 1) 삭제 : 행단위로 1개의 컬럼이라도 NaN이면 삭제
dataDF2.dropna()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare
0,0,3,male,22.0,1,0,A/5 21171,7.2500
1,1,1,female,38.0,1,0,PC 17599,71.2833
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250
3,1,1,female,35.0,1,0,113803,53.1000
4,0,3,male,35.0,0,0,373450,8.0500
...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,382652,29.1250
886,0,2,male,27.0,0,0,211536,13.0000
887,1,1,female,19.0,0,0,112053,30.0000
889,1,1,male,26.0,0,0,111369,30.0000


In [223]:
## 2) 대체 : 전체 데이터의 5%이상 시 삭제 x.
## => 컬럼의 값들 관련성 O : 이전/이후 값을 대체, 주변 여러 값을 기반으로 대체
## => 컬럼의 값들 관련성 X : 결측치가 존재하는 행과 유사한 행들, 관련이 있는 컬럼들 기반으로 대치

## [2-1] 성별에 따른 나이 평균값
display(dataDF2[['Sex', 'Age']].groupby(by='Sex', observed=False).mean())

## [2-2] 등석에 따른 나이 평균값 
display(dataDF2[['Pclass', 'Age']].groupby(by='Pclass', observed=False).mean())

## [2-3] 등석과 성별에 따른 나이 평균값
display(dataDF2[['Pclass','Sex' ,'Age']].groupby(by=['Pclass', 'Sex'], observed=False).median())








,Age
Sex,
female,27.915709
male,30.726645


,Age
Pclass,
1,38.233441
2,29.877630
3,25.140620


Age
Pclass Sex         
1      female  35.0
       male    40.0
2      female  28.0
       male    30.0
3      female  21.5
       male    25.0

In [230]:
## 등석과 성별에 따라 나이 결측치 채우기
d = dataDF2[['Pclass','Sex' ,'Age']].groupby(by=['Pclass', 'Sex'], observed=False).median()
print(d)


mulDF = dataDF2.set_index(['Pclass','Sex'])     ## 멀티 인덱스로 지정
mulDF.fillna(d, inplace=True)                   ## 
mulDF.info()
display('1',mulDF.head(20))

mulDF = mulDF.reset_index()
display('2',mulDF.head(20))

                Age
Pclass Sex         
1      female  35.0
       male    40.0
2      female  28.0
       male    30.0
3      female  21.5
       male    25.0
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 891 entries, (np.int64(3), 'male') to (np.int64(3), 'male')
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Survived  891 non-null    category
 1   Age       891 non-null    float64 
 2   SibSp     891 non-null    int64   
 3   Parch     891 non-null    int64   
 4   Ticket    891 non-null    object  
 5   Fare      891 non-null    float64 
dtypes: category(1), float64(2), int64(2), object(1)
memory usage: 70.2+ KB


'1'

Survived   Age  SibSp  Parch            Ticket     Fare
Pclass Sex                                                           
3      male          0  22.0      1      0         A/5 21171   7.2500
1      female        1  38.0      1      0          PC 17599  71.2833
3      female        1  26.0      0      0  STON/O2. 3101282   7.9250
1      female        1  35.0      1      0            113803  53.1000
3      male          0  35.0      0      0            373450   8.0500
       male          0  25.0      0      0            330877   8.4583
1      male          0  54.0      0      0             17463  51.8625
3      male          0   2.0      3      1            349909  21.0750
       female        1  27.0      0      2            347742  11.1333
2      female        1  14.0      1      0            237736  30.0708
3      female        1   4.0      1      1           PP 9549  16.7000
1      female        1  58.0      0      0            113783  26.5500
3      male          0  20.0      0      0         A/5. 2151   8.0500
       male          0  39.0      1      5            347082  31.2750
       female        0  14.0      0      0            350406   7.8542
2      female        1  55.0      0      0            248706  16.0000
3      male          0   2.0      4      1            382652  29.1250
2      male          1  30.0      0      0            244373  13.0000
3      female        0  31.0      1      0            345763  18.0000
       female        1  21.5      0      0              2649   7.2250

'2'

,Pclass,Sex,Survived,Age,SibSp,Parch,Ticket,Fare
0,3,male,0,22.0,1,0,A/5 21171,7.2500
1,1,female,1,38.0,1,0,PC 17599,71.2833
2,3,female,1,26.0,0,0,STON/O2. 3101282,7.9250
3,1,female,1,35.0,1,0,113803,53.1000
4,3,male,0,35.0,0,0,373450,8.0500
5,3,male,0,25.0,0,0,330877,8.4583
6,1,male,0,54.0,0,0,17463,51.8625
7,3,male,0,2.0,3,1,349909,21.0750
8,3,female,1,27.0,0,2,347742,11.1333
9,2,female,1,14.0,1,0,237736,30.0708


[4] 데이터 분석 진행